In [17]:
import rasterio
from rasterio.windows import Window
from pathlib import Path
from pyproj import Transformer
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
from rasterio.crs import CRS


In [18]:
confirmed_products_dir = Path('confirmed_products')

In [19]:
MGRS_TILE_ID = '52VCH'
MGRS_TILE_ID

'52VCH'

In [20]:
import rasterio
from rasterio.windows import Window
from pyproj import Transformer

def get_pixel_value(tif, lon, lat):
    with rasterio.open(tif) as src:
        transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)
        
        x_utm, y_utm = transformer.transform(lon, lat)
        row, col = src.index(x_utm, y_utm)
        if row < 0 or row >= src.height or col < 0 or col >= src.width:
            raise ValueError("Point is outside the bounds of the raster")
        
        # Create window and read pixel value
        window = Window(col, row, 1, 1)
        val = src.read(1, window=window)[0, 0]
    
    return val

In [21]:
mgrs_ts_dir = confirmed_products_dir / MGRS_TILE_ID
mgrs_ts_prods = sorted(list(mgrs_ts_dir.glob('OPERA*/')))
mgrs_ts_prods[:2]


[PosixPath('confirmed_products/52VCH/OPERA_L3_DIST-ALERT-S1_T52VCH_20240109T092107Z_20251001T081229Z_S1C_30_v0.1'),
 PosixPath('confirmed_products/52VCH/OPERA_L3_DIST-ALERT-S1_T52VCH_20240121T092106Z_20251001T021607Z_S1C_30_v0.1')]

In [22]:
mgrs_ts_status = [prod_dir / f'{prod_dir.name}_GEN-DIST-STATUS.tif' for prod_dir in mgrs_ts_prods]
mgrs_ts_status[:2]

[PosixPath('confirmed_products/52VCH/OPERA_L3_DIST-ALERT-S1_T52VCH_20240109T092107Z_20251001T081229Z_S1C_30_v0.1/OPERA_L3_DIST-ALERT-S1_T52VCH_20240109T092107Z_20251001T081229Z_S1C_30_v0.1_GEN-DIST-STATUS.tif'),
 PosixPath('confirmed_products/52VCH/OPERA_L3_DIST-ALERT-S1_T52VCH_20240121T092106Z_20251001T021607Z_S1C_30_v0.1/OPERA_L3_DIST-ALERT-S1_T52VCH_20240121T092106Z_20251001T021607Z_S1C_30_v0.1_GEN-DIST-STATUS.tif')]

In [23]:
mgrs_ts_dates = [pd.to_datetime(prod_dir.name.split('_')[4]).date() for prod_dir in mgrs_ts_prods]
mgrs_ts_dates[:2]

[datetime.date(2024, 1, 9), datetime.date(2024, 1, 21)]

# Dist Tables

In [24]:
df_val = pd.read_csv('../tables/reference_data/selectedpointsLL.csv')
df_val

,ID,Block,subID,blockStratum,substratum,zone,x,y,centx,centy,long,lat,MGRS
0,30961_1,30961,1,treelossTF,3,33,372932.570128,5.453950e+05,372945,545385,13.854048,4.933158,33NUF
1,30961_2,30961,2,treelossTF,2,33,372728.616472,5.425860e+05,372735,542595,13.852197,4.907920,33NUF
2,30961_3,30961,3,treelossTF,1,33,373846.916077,5.470932e+05,373845,547095,13.862138,4.948639,33NUF
3,30961_4,30961,4,treelossTF,4,33,371390.914129,5.460691e+05,371385,546075,13.839969,4.939375,33NUF
4,30961_5,30961,5,treelossTF,3,33,374504.487544,5.464576e+05,374505,546465,13.868099,4.942951,33NUF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1828,1473302_16,1473302,16,oldcrop_short,2,21,375509.165457,-4.203209e+06,375495,-4203195,-58.417441,-37.967845,21HUU
1829,1473302_17,1473302,17,oldcrop_short,2,21,378407.482557,-4.204245e+06,378405,-4204245,-58.384499,-37.977701,21HUU
1830,1473302_18,1473302,18,oldcrop_short,1,21,374961.521864,-4.197591e+06,374955,-4197585,-58.422612,-37.917224,21HUU
1831,1473302_19,1473302,19,oldcrop_short,4,21,380894.210195,-4.196291e+06,380895,-4196295,-58.354839,-37.906398,21HUU


In [25]:
df_val_mgrs = df_val[df_val['MGRS'] == MGRS_TILE_ID].reset_index(drop=True)
sites_in_mgrs_tile = df_val_mgrs.to_dict(orient='records')
sites_in_mgrs_tile[:2]

[{'ID': '557332_1',
  'Block': 557332,
  'subID': 1,
  'blockStratum': 'fire',
  'substratum': 3,
  'zone': 52,
  'x': 361550.14969268,
  'y': 6256802.33049892,
  'centx': 361545,
  'centy': 6256815,
  'long': 126.754486406887,
  'lat': 56.4355224187291,
  'MGRS': '52VCH'},
 {'ID': '557332_2',
  'Block': 557332,
  'subID': 2,
  'blockStratum': 'fire',
  'substratum': 4,
  'zone': 52,
  'x': 359165.724007643,
  'y': 6262049.51549871,
  'centx': 359175,
  'centy': 6262035,
  'long': 126.713267950363,
  'lat': 56.4816818888803,
  'MGRS': '52VCH'}]

In [26]:
tables = []
for site in tqdm(sites_in_mgrs_tile[:], desc='Processing sites'):
    site_id = site['ID']
    site_lon = site['long']
    site_lat = site['lat'] 

    labels = [get_pixel_value(tif, site_lon, site_lat) for tif in mgrs_ts_status]

    df_dist_s1_table_site = pd.DataFrame({
        'site_id': [site_id] * len(labels),
        'date': mgrs_ts_dates,
        # 'longitude': [site_lon] * len(labels),
        # 'latitude': [site_lat] * len(labels),
        'labels': labels,
    })
    tables.append(df_dist_s1_table_site)

tables[0].head()


Processing sites: 100%|█████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 16.91it/s]


,site_id,date,labels
0,557332_1,2024-01-09,0
1,557332_1,2024-01-21,0
2,557332_1,2024-02-02,0
3,557332_1,2024-02-26,0
4,557332_1,2024-03-09,0


In [27]:
tables[1].head(20)


,site_id,date,labels
0,557332_2,2024-01-09,0
1,557332_2,2024-01-21,1
2,557332_2,2024-02-02,2
3,557332_2,2024-02-26,2
4,557332_2,2024-03-09,2
5,557332_2,2024-03-21,3
6,557332_2,2024-04-02,3
7,557332_2,2024-04-14,3
8,557332_2,2024-04-26,3
9,557332_2,2024-05-08,3


In [28]:
for table in tables:
    site_id = table['site_id'].tolist()[0]
    block_id = site_id.split('_')[0]
    table_dir = Path(f'dist_s1_label_tables/{block_id}')
    table_dir.mkdir(exist_ok=True, parents=True)
    table_path = table_dir / f'{site_id}.csv'
    table.to_csv(table_path, index=False)


# Generate Site Vector Table

In [29]:
df_site_geo = gpd.GeoDataFrame(df_val_mgrs[['ID']],
                               geometry=gpd.points_from_xy(df_val_mgrs.long, 
                                                           df_val_mgrs.lat), 
                               crs=CRS.from_epsg(4326))
df_site_geo.head()


,ID,geometry
0,557332_1,POINT (126.75449 56.43552)
1,557332_2,POINT (126.71327 56.48168)
2,557332_3,POINT (126.66233 56.50176)
3,557332_4,POINT (126.67221 56.44398)
4,557332_5,POINT (126.71327 56.44933)


In [30]:
df_site_geo.to_file(table_dir / 'sites.geojson', driver='GeoJSON')